#### Create Database Connection and Set Configurations

In [3]:
%%capture
%load_ext sql
%sql sqlite:///northwind.db

In [4]:
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.displaylimit = 100

In [6]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

name,type
Categories,table
sqlite_sequence,table
CustomerCustomerDemo,table
CustomerDemographics,table
Customers,table
Employees,table
EmployeeTerritories,table
Order Details,table
Orders,table
Products,table


#### INTRODUCTION
This analysis project is a guided project provided by Dataquest. The data used in this project derived from Northwind Database (link: https://github.com/pthom/northwind_psql/tree/master)

The analysis for this dataset aims to answer questions relating to : 
1. Employee sales performance ranking
2. Running total of monthly sales
3. Month-over-Month sales growth 
4. High value customers identification
5. Percentage of sales for each product category
6. Top product for each category 

1. Employee ranking based on total sales

In [6]:
%%sql
WITH employee_sales AS (
 SELECT e.employeeid, 
       e.firstname, 
       e.lastname,
       COUNT(od.orderid) AS total_order,
       SUM((odd.quantity*odd.unitprice)-odd.discount) AS total_sales
  FROM orders AS od
  JOIN [order details] AS odd 
    ON od.orderid=odd.orderid
  JOIN employees AS e
    ON od.employeeid=e.employeeid
 GROUP BY e.employeeid)
 
 SELECT *,
        RANK () OVER (ORDER BY total_sales DESC) AS rank
   FROM employee_sales;

employeeid,firstname,lastname,total_order,total_sales,rank
4,Margaret,Peacock,420,250161.69999999992,1
3,Janet,Leverling,321,213035.34999999992,2
1,Nancy,Davolio,345,202126.72,3
2,Andrew,Fuller,241,177738.71000000008,4
7,Robert,King,176,141283.04000000004,5
8,Laura,Callahan,260,133286.42999999996,6
9,Anne,Dodsworth,107,82956.7,7
6,Michael,Suyama,168,78188.95000000001,8
5,Steven,Buchanan,117,75559.95,9


#### Visualization on best-performing employees : 

https://public.tableau.com/app/profile/novita.eliana/viz/NorthwindTraders-1_EmployeeRankings/NorthwindTraders-1_Employeerankings

#### INSIGHT 
Based on total sales, the top-performing employee is Margaret Peacock with total sales of 250161 and the least performing employee is Steven Buchanan with total sales of 75559. 

#### RECOMMENDATION
Based on the insight provided above, company's management can recognize and reward top-performing employee and provide necessary support for the least-performing employee.

#### 2. Running total of monthly sales

In [5]:
%%sql 

WITH monthly_sales AS ( 
SELECT strftime ('%Y',od.orderdate) AS year, 
       strftime ('%m',od.orderdate) AS month, 
       SUM(odd.unitprice * odd.quantity * (1 - odd.discount)) AS total_monthly_sales
  FROM orders AS od
  JOIN [order details] AS odd
    ON od.orderid=odd.orderid
    GROUP BY strftime ('%Y',od.orderdate), strftime ('%m',od.orderdate)
     ORDER BY year,month)
    
    SELECT *, 
           SUM(total_monthly_sales) OVER (PARTITION BY year ORDER BY year ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total 
      FROM monthly_sales;

year,month,total_monthly_sales,running_total
2016,07,27861.894999999997,27861.894999999997
2016,08,25485.275000000005,53347.17
2016,09,26381.399999999998,79728.56999999999
2016,10,37515.72499999999,117244.29499999998
2016,11,45600.045,162844.33999999997
2016,12,45239.63000000001,208083.96999999997
2017,01,61258.07,61258.07
2017,02,38483.635,99741.705
2017,03,38547.219999999994,138288.925
2017,04,53032.9525,191321.8775


#### INSIGHT 
The data above show a sales trend over time. Monitoring company's overall sales performance periodically can help company understand sales trend better and devise sales strategy accordingly.

3. Monthly sales growth

In [3]:
%%sql

WITH monthly_sales AS ( 
SELECT strftime ('%Y',od.orderdate) AS year, 
       strftime ('%m',od.orderdate) AS month, 
       SUM(odd.unitprice * odd.quantity * (1 - odd.discount)) AS current_month_sales
  FROM orders AS od
  JOIN [order details] AS odd
    ON od.orderid=odd.orderid
    GROUP BY strftime ('%Y',od.orderdate), strftime ('%m',od.orderdate)
     ORDER BY year,month),
    
previous_month_sales AS (
SELECT *, 
        LAG(current_month_sales) OVER (PARTITION BY year ORDER BY month) AS last_month_sales 
   FROM monthly_sales),
   
growth_rate_table AS (
SELECT *, 
         (current_month_sales-last_month_sales)/last_month_sales*100 AS growth_rate
    FROM previous_month_sales)
    
SELECT *, 
       CASE
            WHEN growth_rate>0 THEN 'Increase'
            WHEN growth_rate<0 THEN 'Decrease'
            ELSE 'No Change'
             END AS growth_change
  FROM growth_rate_table;



year,month,current_month_sales,last_month_sales,growth_rate,growth_change
2016,07,27861.894999999997,None,None,No Change
2016,08,25485.275000000005,27861.894999999997,-8.530001279525287,Decrease
2016,09,26381.399999999998,25485.275000000005,3.5162461460588226,Increase
2016,10,37515.72499999999,26381.399999999998,42.20520897298852,Increase
2016,11,45600.045,37515.72499999999,21.549150389603316,Increase
2016,12,45239.63000000001,45600.045,-0.7903829919465789,Decrease
2017,01,61258.07,None,None,No Change
2017,02,38483.635,61258.07,-37.17785264863878,Decrease
2017,03,38547.219999999994,38483.635,0.16522607596707495,Increase
2017,04,53032.9525,38547.219999999994,37.57918858999432,Increase


#### Visualization on monthly growth rate : 

https://public.tableau.com/app/profile/novita.eliana/viz/NorthwindTraders-2_MonthlyGrowthRate/NorthwindTraders-2_MonthlyGrowthRate

#### INSIGHT
The data above provide a more detailed picture on monthly sales growth, informing company the pattern of increasing and decreasing sales from month to month. The insight can be used to help company devise an appropriate sales strategy to achieve sales goal. 

4. High value customer identification

In [15]:
%%sql

WITH customer_order_value AS (
    SELECT c.customerid, 
           COUNT(od.orderid) AS total_order, 
           SUM(odd.unitprice * odd.quantity * (1 - odd.discount)) AS total_order_value
      FROM orders AS od
      JOIN [order details] AS odd
        ON od.orderid=odd.orderid
      JOIN customers AS c
        ON od.customerid=c.customerid
     GROUP BY c.customerid)
   
   SELECT *, 
          CASE
          WHEN total_order_value > AVG(total_order_value) OVER()THEN 'Above Average' 
          ELSE 'Below Average'                                
           END AS customer_category                      
     FROM customer_order_value;



customerid,total_order,total_order_value,customer_category
ALFKI,12,4273.0,Below Average
ANATR,10,1402.95,Below Average
ANTON,17,7023.977499999999,Below Average
AROUT,30,13390.650000000001,Below Average
BERGS,52,24927.577499999996,Above Average
BLAUS,14,3239.8,Below Average
BLONP,26,18534.08,Above Average
BOLID,6,4232.85,Below Average
BONAP,44,21963.252500000002,Above Average
BOTTM,35,20801.6,Above Average


#### Visualization on list of high value customers : 

https://public.tableau.com/app/profile/novita.eliana/viz/NorthwindTraders-3_HighValueCustomers/NorthwindTraders-3_HighValueCustomer

#### INSIGHT 
The data outlines high-value and below average customers based on their total order value. Company can use this insight to tailor  promotion and offers that could drive sales and improve customers engagement. 

5. Percentage of sales for each product category

In [16]:
%%sql

WITH product_category_sales AS (
    SELECT c.categoryid, 
           c.categoryname, 
           SUM(od.unitprice * od.quantity * (1 - od.discount)) AS total_sales
      FROM categories AS c
      JOIN products AS p
        ON c.categoryid = p.categoryid
      JOIN [order details] AS od
        ON p.productid = od.productid
    GROUP BY c.categoryid)
    
 SELECT *, 
        total_sales/SUM (total_sales) OVER ()*100 AS percentage_total_sales
   FROM product_category_sales
   ORDER BY percentage_total_sales DESC;


#### Visualization on percentage of sales for each product category :

https://public.tableau.com/app/profile/novita.eliana/viz/NorthwindTraders-4_PercentageofSalesforEachProductCategory/NorthwindTraders-4_PercentageofSalesEachProductCategory

#### INSIGHT
The data above show that beverages, dairy products and condections accounted for more than 50% of company's sales. Understanding percentage of each product category will help company  managing inventory as well as improving marketing strategies.

6. Top product per category

In [21]:
%%sql

WITH product_category_sales AS (
    SELECT c.categoryid, 
           c.categoryname,
           p.productid,
           p.productname,
           SUM(od.unitprice*od.quantity*(1-discount)) AS total_sales
      FROM categories AS c
      JOIN products AS p
        ON c.categoryid = p.categoryid
      JOIN [order details] AS od
        ON p.productid = od.productid
    GROUP BY c.categoryid,p.productid),
    
ranking_table AS (SELECT *, 
           ROW_NUMBER () OVER (PARTITION BY categoryid ORDER BY total_sales DESC) AS rank 
      FROM product_category_sales)
      
SELECT *
  FROM ranking_table
 WHERE rank<=3;



categoryid,categoryname,productid,productname,total_sales,rank
1,Beverages,38,Côte de Blaye,141396.735,1
1,Beverages,43,Ipoh Coffee,23526.699999999997,2
1,Beverages,2,Chang,16355.96,3
2,Condiments,63,Vegie-spread,16701.095,1
2,Condiments,61,Sirop d'érable,14352.6,2
2,Condiments,65,Louisiana Fiery Hot Pepper Sauce,13869.890000000001,3
3,Confections,62,Tarte au sucre,47234.97,1
3,Confections,20,Sir Rodney's Marmalade,22563.359999999997,2
3,Confections,26,Gumbär Gummibärchen,19849.144500000002,3
4,Dairy Products,59,Raclette Courdavault,71155.7,1


#### Visualization on top product per category : 

https://public.tableau.com/app/profile/novita.eliana/viz/NorthwindTraders-5_TopProductsinEachCategory/NorthwindTraders-5_Percentageof

#### INSIGHT
The data show the top 3 most-sold products for each category. This insight can be used by company to identify star product and ensure their availibility in invetory and keep their popularity through marketing efforts. 